In [1]:
include("JQlaw.jl")
using .JQlaw
using Plotly

WebIO._IJuliaInit()

## Simple Orbit Raise Example

In [2]:
orbit_inital = KeplarianOrbit(
                450000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

orbit_target = KeplarianOrbit(
                600000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.5,
                0,
                10, 
                500.,
                10^6,
                10^3)

Sat_Params = sat_params(100, 0.8, 0.5/100.)

time_initial = 59715.0 # MJD

Q_hist, orbit_hist, time_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)


# # store ECI terms
# ECI_x = []
# ECI_y = []
# ECI_z = []
# for orbit in orbit_history
#     equinoctial_orbit = EquinoctialOrbit(orbit[1], orbit[2], orbit[3], orbit[4], orbit[5], orbit[6])
#     ECI_orbit = Equinoctial2ECI(equinoctial_orbit)
#     push!(ECI_x, ECI_orbit[1])
#     push!(ECI_y, ECI_orbit[2])
#     push!(ECI_z, ECI_orbit[3])
# end

5.236278816394392e8
4.9129088147076845e8
4.632591877056975e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.2890594618124384e8
4.010049728115263e8
3.721243415439715e8
3.451618093167987e8
3.2311813206295514e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
2.955420846264036e8
2.7159092926174015e8
2.481395947900941e8
2.2722551148817983e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.0336768140770242e8
1.850331583078985e8
1.6591245497602487e8
1.4815484582072031e8
1.3369690552500097e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.166883079127281e8
1.0213019730868906e8
8.813016354277432e7
7.5918379

(Any[5.580870563371612e8, 5.236278816394392e8, 4.9129088147076845e8, 4.632591877056975e8, 4.452469279371275e8, 4.452469279371275e8, 4.452469279371275e8, 4.452469279371275e8, 4.452469279371275e8, 4.452469279371275e8  …  2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 2683.2280417613406, 980.0255737755431], Any[[6.820993179e6, 0.0009999999993907651, 3.4906585032797823e-8, 8.726646258864029e-6, 1.52308709889488e-10, 3.490658503988659e-5], [6.825451132347688e6, 0.0016203708906017877, 0.0001790933306615142, 8.726646258864029e-6, 1.5230870988936938e-10, 0.5615408195167376], [6.8297868113640975e6, 0.0020953132589541378, 0.000597221561584746, 8.72664621628494e-6, 1.522609950413583e-10, 1.1227651754531878], [6.83367891427862e6, 0.002346135954797809, 0.0011263859533824291, 8.72664615932306e-6, 1.519245390018999e-10, 1.68313598722623], [6.836242701205792e6, 0.0024498443755707096, 0.0015